In [1]:
import pandas as pd

## Filtering excel sheet

In [ ]:
df = pd.read_excel('./Gringotts.xlsx')
excel_file = pd.ExcelFile('./Gringotts.xlsx')

df['Numbers'] = df['College'].str.extract(r'(\d+\.\d+\s\d+\.\d+\s\d+\.\d+\s\d+\.\d+\s\d+\.\d+)$')
df[['TLR', 'RPC', 'GO', 'OI', 'PR']] = df['Numbers'].str.split().apply(pd.Series)
df.drop(columns=['Numbers'], inplace=True)

In [29]:
import re
def extract_and_calculate_numbers(text):
  # numbers = re.findall(r'\d+\.\d+', text)
  numbers = re.findall(r'\d+(?:\.\d+)?', text)
  numbers = numbers[5:]
  if len(numbers) >= 5:
      tlr = float(numbers[0])
      rpc = float(numbers[1])
      go = float(numbers[2])
      oi = float(numbers[3])
      pr = float(numbers[4])
  else:
      tlr, rpc, go, oi, pr = None, None, None, None, None
  
  return pd.Series({'TLR': tlr, 'RPC': rpc, 'GO': go, 'OI': oi, 'PR': pr})


excel_file = pd.ExcelFile('./Gringotts.xlsx')
df = pd.read_excel(excel_file, sheet_name="2016")
df[['TLR', 'RPC', 'GO', 'OI', 'PR']] = df['Name'].apply(extract_and_calculate_numbers)
df

# df['Numbers'] = df['Name'].str.extract(r'(\d+\.\d+\s\d+\.\d+\s\d+\.\d+\s\d+\.\d+\s\d+\.\d+)$')
# df[['TLR', 'RPC', 'GO', 'OI', 'PR']] = df['Numbers'].str.split().apply(pd.Series)
# df.drop(columns=['Numbers'], inplace=True)
df.to_excel('nirf/2016.xlsx', index=False)

In [2]:
df = pd.read_excel('./nirf/2021.xlsx')
df['Name'] = df['Name'].str.split('Close', expand=True)[0]
df.to_excel('nirf/2021.xlsx')

## Adding dummy data

In [2]:
subtopic_weights = {
    "TLR": {"SS": 20, "FSR": 30, "FQE": 20, "FRU": 30},
    "RPC": {"PU": 35, "QP": 40, "IPR": 15, "FPPP": 10},
    "GO": {"GPH": 40, "GUE": 15, "GMS": 25, "GPHD": 20},
    "OI": {"RD": 30, "WD": 30, "ESCS": 20, "PCS": 20},
}

In [31]:
df = pd.read_excel('./nirf/xl/2021.xlsx')
def add_dummy_data(df, property_name):
    subtopics = list(subtopic_weights[property_name].keys())
    total_value = df[property_name].values[0]

    for subtopic in subtopics:
        percentage = subtopic_weights[property_name][subtopic]
        dummy_value = (percentage / 100) * total_value
        df[subtopic] = dummy_value

for property in subtopic_weights.keys():
    add_dummy_data(df, property)

df.to_csv('nirf/2021.csv')

In [39]:
df = pd.read_csv('./nirf/2023.csv')
df.isna().any().any()

False

## Rank prediction

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [3]:
weightage = {
  'TLR': 0.30, 
  'RPC': 0.30, 
  'GO': 0.20, 
  'OI': 0.10, 
  'PR': 0.10
}

In [32]:
df = pd.read_csv('./nirf/2020.csv')
# df = pd.read_csv("C:/Users/piyus/Downloads/archive/engineering.csv")

df['Weighted_Score'] = (
  df['TLR'] * weightage['TLR'] +
  df['RPC'] * weightage['RPC'] +
  df['GO'] * weightage['GO'] +
  df['OI'] * weightage['OI'] +
  df['PR'] * weightage['PR']
)
df[90:]

,Institute ID,Name,City,State,Score,Rank,TLR,RPC,GO,OI,...,FPPP,GPH,GUE,GMS,GPHD,RD,WD,ESCS,PCS,Weighted_Score
90,IR-E-C-1297,Siddaganga Institute of Technology,Tumkur,Karnataka,38.23,91,60.33,13.44,54.58,50.17,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,38.227
91,IR-E-U-0072,National Institute of Technology Patna,Patna,Bihar,38.21,92,52.49,16.60,59.09,43.42,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,38.207
92,IR-E-C-30045,C.V. Raman Global University,Bhubaneswar,Odisha,37.94,93,59.49,5.07,58.70,57.65,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,37.943
93,IR-E-U-0733,PES University,Bengaluru,Karnataka,37.94,93,57.56,12.20,55.31,49.22,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,37.939
94,IR-E-U-0489,Vel Tech Rangarajan Dr. Sagunthala R & D Insti...,Chennai,Tamil Nadu,37.86,95,57.70,24.09,36.36,59.88,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,37.852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,IR-E-C-42227,Dr. D. Y. Patil Institute of Technology,Pune,Maharashtra,31.15,196,49.69,1.66,53.63,48.50,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,31.144
196,IR-E-C-49660,Pimpri Chinchwad College of Engineering,Pune,Maharashtra,31.13,197,46.11,7.36,50.73,48.76,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,31.118
197,IR-E-C-34167,Ramrao Adik Institute of Technology,Navi Mumbai,Maharashtra,31.11,198,53.60,3.82,45.75,47.25,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,31.101
198,IR-E-C-48145,BVRIT Hyderabad,Hyderabad,Telangana,31.10,199,49.16,0.46,53.74,52.98,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,31.095


In [18]:
X = df[['Weighted_Score']]
y = df['Rank']

In [19]:
model2 = RandomForestRegressor(n_estimators=100, random_state=0)
model1 = LinearRegression()
model1.fit(X, y)
model2.fit(X, y)

RandomForestRegressor(random_state=0)

In [29]:
df1 = pd.read_csv('./nirf/2023.csv')
scores = {'TLR': 30, 'RPC': 50, 'GO': 30, 'OI': 70, 'PR': 10}
scores = pd.DataFrame(scores, index=[0])
scores['Weighted_Score'] = (
  scores['TLR'] * weightage['TLR'] +
  scores['RPC'] * weightage['RPC'] +
  scores['GO'] * weightage['GO'] +
  scores['OI'] * weightage['OI'] +
  scores['PR'] * weightage['PR']
)
predictions = model1.predict(scores[['Weighted_Score']])
scores['Predicted_Rank1'] = predictions
# scores['Predicted_Rank1_Round'] = scores['Predicted_Rank1'].apply(round)

predictions = model2.predict(scores[['Weighted_Score']])
scores['Predicted_Rank2'] = predictions
# scores['Predicted_Rank2_Round'] = scores['Predicted_Rank2'].apply(round)
# df1 = df1.sort_values(by='Predicted_Rank')
# df1.reset_index(drop=True, inplace=True)
scores
# current_year_data = current_year_data.sort_values(by='Predicted_Rank')

# rmse = np.sqrt(mean_squared_error(y, model2.predict(X)))
# rmse

,TLR,RPC,GO,OI,PR,Weighted_Score,Predicted_Rank1,Predicted_Rank2
0,30,50,30,70,10,38.0,116.485187,93.26


In [40]:
df = pd.read_csv('./nirf/2016.csv')
# df.to_dict(orient='records')[0]
df = df.drop(columns=['TLR', 'RPC', 'GO', 'OI', 'PR', 'SS', 'FSR', 'FQE', 'FRU', 'PU', 'QP', 'IPR', 'FPPP', 'GPH', 'GUE', 'GMS', 'GPHD', 'RD', 'WD', 'ESCS', 'PCS'])
df

,Institute ID,Name,City,State,Score,Rank
0,NIRF-ENGG-INF-77,"Indian Institute Of Technology, Madras",Chennai,Tamil Nadu,89.410,1
1,NIRF-ENGG-INF-312,"Indian Institute Of Technology, Bombay",Bombay,Maharashtra,87.660,2
2,NIRF-ENGG-INF-300,"Indian Institute Of Technology, Kharagpur",Kharagpur,West Bengal,83.910,3
3,NIRF-ENGG-INF-79,"Indian Institute Of Technology, Delhi",New Delhi,Delhi,82.020,4
4,NIRF-ENGG-INF-228,"Indian Institute Of Technology, Kanpur",Kanpur,Uttar Pradesh,81.070,5
...,...,...,...,...,...,...
95,NIRF-ENGG-1-2455794950,The National Institute Of Engineering,MYSORE,Karnataka,49.514,96
96,NIRF-ENGG-1-2453623896,"Government College Of Engineering, Aurangabad ...",AURANGABAD,Maharashtra,49.511,97
97,NIRF-ENGG-1-2449945774,Anand Institute Of Higher Technology,CHENNAI,Tamil Nadu,49.410,98
98,NIRF-ENGG-1-2454813228,Noida Institute Of Engineering & Technology,GAUTAM BUDH NAGAR,Uttar Pradesh,49.240,99
